<a href="https://colab.research.google.com/github/jyotidabass/Evaluating-LLMs-Benchmarks-evaluation-framework-and-popular-leaderboards/blob/main/Evaluating_LLMs_Benchmarks%2C_evaluation_framework_and_popular_leaderboards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
!pip install datasets
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3) # Assuming 3 labels for the task
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load the benchmark data
benchmark_data = pd.read_parquet("hf://datasets/nyu-mll/glue/ax/test-00000-of-00001.parquet")

# Convert pandas DataFrame to Hugging Face Dataset
benchmark_dataset = Dataset.from_pandas(benchmark_data)

# Define a function to preprocess the data for the model
def preprocess_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], padding="max_length", truncation=True)

# Preprocess the dataset
encoded_dataset = benchmark_dataset.map(preprocess_function, batched=True)

# --- MODIFICATION START ---
# Replace -1 values in 'label' column with a valid label (e.g., 0)
encoded_dataset = encoded_dataset.map(lambda examples: {'label': [0 if label == -1 else label for label in examples['label']]}, batched=True)
# --- MODIFICATION END ---


# Define Trainer arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,  # Adjust batch size as needed
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=encoded_dataset,
)

# Evaluate the model
results = trainer.evaluate()

# Access specific metrics from the results dictionary
print(results)  # Print the evaluation results

print(results['eval_loss'])  # Access accuracy, for example

# Assuming 'leaderboard' is defined elsewhere and 'submit' method accepts a dictionary
# Placeholder for leaderboard object, replace with your implementation:
class Leaderboard:
    def submit(self, results):
        print("Submitting results to leaderboard:", results)

leaderboard = Leaderboard()
leaderboard.submit(results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1104 [00:00<?, ? examples/s]

Map:   0%|          | 0/1104 [00:00<?, ? examples/s]

{'eval_loss': 1.192099928855896, 'eval_model_preparation_time': 0.0055, 'eval_runtime': 29.7538, 'eval_samples_per_second': 37.105, 'eval_steps_per_second': 4.638}
1.192099928855896
Submitting results to leaderboard: {'eval_loss': 1.192099928855896, 'eval_model_preparation_time': 0.0055, 'eval_runtime': 29.7538, 'eval_samples_per_second': 37.105, 'eval_steps_per_second': 4.638}


In [5]:
print(results['eval_loss'])  # Access accuracy, for example

# Assuming 'leaderboard' is defined elsewhere and 'submit' method accepts a dictionary
# Placeholder for leaderboard object, replace with your implementation:
class Leaderboard:
    def submit(self, results):
        print("Submitting results to leaderboard:", results)

leaderboard = Leaderboard()
leaderboard.submit(results)

1.7214229106903076
Submitting results to leaderboard: {'eval_loss': 1.7214229106903076, 'eval_model_preparation_time': 0.0033, 'eval_runtime': 30.5611, 'eval_samples_per_second': 36.124, 'eval_steps_per_second': 4.516}
